# Passos

# Objetivo: Prever próximos 15 dias

# 1°Coleta e Preparação dos Dados

## Analisar dados importantes

## Procurar por valores ausentes e outliers

## Realizar a conversão e manipulação de dados

# 2°Análise Exploratória de Dados (EDA)

## Visualizar os dados.

## Verificar a estacionaridade e sazonalidade.

## Analisar os padrões, tendências e ciclos.

 

In [ ]:
import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px


In [ ]:
from statsmodels.stats.power import tt_ind_solve_power
from scipy.stats import ks_2samp
from scipy.stats import spearmanr
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_harvey_collier
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score
from statsmodels.tsa.stattools import adfuller
#statistics libraries
import scipy
from statsmodels.tools.eval_measures import rmse
from statsmodels.graphics.tsaplots import month_plot, seasonal_plot, plot_acf, plot_pacf, quarter_plot
from statsmodels.stats.diagnostic import acorr_ljungbox as ljung
#from nimbusml.timeseries import SsaForecaster
from statsmodels.tsa.statespace.tools import diff as diff
from scipy import signal
from scipy.stats import shapiro
from scipy.stats import boxcox
import scipy.stats as stats
from scipy.stats import jarque_bera as jb
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_error

from sklearn.model_selection import train_test_split


In [ ]:

dataframes = ["holidays_events.csv", "oil.csv", "sample_submission.csv", "stores.csv", "test.csv", "train.csv", "transactions.csv"]

for i in dataframes:
    # Retirar a extensão do nome do arquivo para usar como nome da variável
    var_name = i.split('.')[0]
    # Utilizar globals() para criar uma variável com esse nome e atribuir o DataFrame a ela
    globals()[var_name] = pd.read_csv(i)
    print(globals()[var_name].head())
    print()
    print(globals()[var_name].info())
    print()
    print(globals()[var_name].describe())


# Train

In [ ]:
train

In [ ]:
train.info()

In [ ]:
train['date']=pd.to_datetime(train['date'])

In [ ]:
train.describe()

In [ ]:
effect_size = 0.1  # Tamanho do efeito desejado
alpha = 0.05       # Nível de significância
power = 0.95       # Poder desejado

sample_size = tt_ind_solve_power(effect_size=effect_size, alpha=alpha, power=power)

print(f"Tamanho da amostra necessário: {sample_size}")

# Tamanho suficiente

In [ ]:
holidays_events

sns.histplot(train['sales'])
plt.show()

In [ ]:
train[train.isnull().any(axis=1)]

In [ ]:
train['sales'].quantile([0.25, 0.5, 0.75, 1])

In [ ]:
plt.hist(train['sales'], bins=50, edgecolor='black')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.title('Histogram of Sales')
plt.show()

In [ ]:
plt.hist(train['sales'], bins=50, edgecolor='black')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.title('Histogram of Sales')
plt.show()

# Analisando transactions

In [ ]:
transactions.info()

In [ ]:
transactions['date']=pd.to_datetime(transactions['date'])

In [ ]:
temp = pd.merge(train.groupby(["date", "store_nbr"]).sales.sum().reset_index(), transactions, how = "left")
print("Correlação de Spearman de vendas totais e transações {:,.4f}".format(temp.corr("spearman").sales.loc["transactions"]))
px.line(transactions.sort_values(["store_nbr", "date"]), x='date', y='transactions', color='store_nbr',title = "Transactions" )

In [ ]:
a = transactions.copy()
a["year"] = a.date.dt.year
a["month"] = a.date.dt.month
px.box(a, x="year", y="transactions" , color = "month", title = "Transactions")

In [ ]:
a = transactions.set_index("date").resample("M").transactions.mean().reset_index()
a["year"] = a.date.dt.year
px.line(a, x='date', y='transactions', color='year',title = "Monthly Average Transactions" )

In [ ]:
px.scatter(temp, x = "transactions", y = "sales", trendline = "ols", trendline_color_override = "red")


In [ ]:
a = transactions.copy()
a["year"] = a.date.dt.year
a["dayofweek"] = a.date.dt.dayofweek+1
a = a.groupby(["year", "dayofweek"]).transactions.mean().reset_index()
px.line(a, x="dayofweek", y="transactions" , color = "year", title = "Transactions")

É observável uma relação de sazonalidade tanto mensal, como semanal também.

# Dando uma olhada nos dfs de feriados e óleo

In [ ]:
oil.info()

In [ ]:
oil['date']=pd.to_datetime(oil['date'])

In [ ]:
oil.head()

In [ ]:
plt.hist(oil['dcoilwtico'], bins=50, edgecolor='black')
plt.xlabel('Oil Price')
plt.ylabel('Frequency')
plt.title('Histogram of Oil Price')
plt.show()

In [ ]:
# Interpolate
oil["dcoilwtico"] = np.where(oil["dcoilwtico"] == 0, np.nan, oil["dcoilwtico"])
oil["dcoilwtico_interpolated"] =oil.dcoilwtico.interpolate()
# Plot
p = oil.melt(id_vars=['date']+list(oil.keys()[5:]), var_name='Legend')
px.line(p.sort_values(["Legend", "date"], ascending = [False, True]), x='date', y='value', color='Legend',title = "Daily Oil Price" )

In [ ]:
temp = pd.merge(temp, oil, how = "left")
print("Correlation with Daily Oil Prices")
print(temp.drop(["store_nbr", "dcoilwtico"], axis = 1).corr("spearman").dcoilwtico_interpolated.loc[["sales", "transactions"]], "\n")


fig, axes = plt.subplots(1, 2, figsize = (15,5))
temp.plot.scatter(x = "dcoilwtico_interpolated", y = "transactions", ax=axes[0])
temp.plot.scatter(x = "dcoilwtico_interpolated", y = "sales", ax=axes[1], color = "r")
axes[0].set_title('Daily oil price & Transactions', fontsize = 15)
axes[1].set_title('Daily Oil Price & Sales', fontsize = 15);

# Agora observando os feriados

In [ ]:
holidays_events.info()

In [ ]:
holidays_events['date']=pd.to_datetime(holidays_events['date'])

In [ ]:
holidays_events=holidays_events[holidays_events['transferred']==False]

In [ ]:
holidays_events

# Analisando eventos que podem afetar a análise.

In [ ]:
train['payday'] = ((train['date'].dt.day == 15) | (train['date'].dt.is_month_end)).astype(int)

In [ ]:
earthquake_date = pd.Timestamp('2016-04-16')
train['earthquake_effect'] = ((train['date'] > earthquake_date) & (train['date'] < earthquake_date + pd.Timedelta(weeks=4))).astype(int)

In [ ]:
train = train.merge(oil, on='date', how='left')

In [ ]:
test['date']=pd.to_datetime(test['date'])
test=test.merge(oil, on='date', how='left')
test['payday'] = ((test['date'].dt.day == 15) | (test['date'].dt.is_month_end)).astype(int)
test['dcoilwtico'] = test['dcoilwtico'].fillna(method='ffill')
test['holiday'] = test['date'].isin(holidays_events['date']).astype(int)


In [ ]:
train['dcoilwtico'] = train['dcoilwtico'].fillna(method='ffill')


In [ ]:
train['holiday'] = train['date'].isin(holidays_events['date']).astype(int)


# Procurando correlação

In [ ]:
sns.heatmap(train[['sales', 'onpromotion', 'payday', 'holiday', 'dcoilwtico']].corr(), annot=True)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='payday', y='sales', data=train)
plt.title('Vendas fora dos dias do pagamento e dentro deles')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='holiday', y='sales', data=train)
plt.title('Vendas nos fora dos feriados e dentro deles')
plt.show()

In [ ]:
# Agrupando os dados por data e somando as vendas para cada dia
daily_sales = train.groupby('date').agg({
    'sales': 'sum',
    'dcoilwtico': 'mean'  # Presumindo que o preço do óleo seja o mesmo para todo o dia, ou você pode escolher outra forma de agregá-lo.
}).reset_index()

plt.figure(figsize=(10,6))
sns.scatterplot(x=daily_sales['dcoilwtico'], y=daily_sales['sales'])
plt.xlabel('Preço do óleo')
plt.ylabel('Vendas Diárias Totais')
plt.title('Relação entre o Preço do Óleo e as Vendas Diárias Totais')
plt.show()

Parece que quanto maior o valor do preço do óleo, menor o número de vendas totais. Talvez a alta no preço do óleo cause inflação nos produtos e acabe diminuindo as vendas totais.

# Analisando vendas totais

In [ ]:
a = train[["store_nbr", "sales"]]
a["ind"] = 1
a["ind"] = a.groupby("store_nbr").ind.cumsum().values
a = pd.pivot(a, index = "ind", columns = "store_nbr", values = "sales").corr()
mask = np.triu(a.corr())
plt.figure(figsize=(20, 20))
sns.heatmap(a,
        annot=True,
        fmt='.1f',
        cmap='coolwarm',
        square=True,
        mask=mask,
        linewidths=1,
        cbar=False)
plt.title("Correlations among stores",fontsize = 20)
plt.show()

In [ ]:
a = train.set_index("date").groupby("store_nbr").resample("D").sales.sum().reset_index()
px.line(a, x = "date", y= "sales", color = "store_nbr", title = "Daily total sales of the stores")

## Analisei as lojas que demoraram abrir.

In [ ]:
print(train.shape)
train = train[~((train.store_nbr == 52) & (train.date < "2017-04-20"))]
train = train[~((train.store_nbr == 22) & (train.date < "2015-10-09"))]
train = train[~((train.store_nbr == 42) & (train.date < "2015-08-21"))]
train = train[~((train.store_nbr == 21) & (train.date < "2015-07-24"))]
train = train[~((train.store_nbr == 29) & (train.date < "2015-03-20"))]
train = train[~((train.store_nbr == 20) & (train.date < "2015-02-13"))]
train = train[~((train.store_nbr == 53) & (train.date < "2014-05-29"))]
train = train[~((train.store_nbr == 36) & (train.date < "2013-05-09"))]
train.shape

Vou procurar pelas lojas que nunca venderam nada, para usar 0 nos valores de forecasting.

In [ ]:
c = train.groupby(["store_nbr", "family"]).sales.sum().reset_index().sort_values(["family","store_nbr"])
c = c[c.sales == 0]
c

In [ ]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'datetime64[ns]' and col_type.name != 'category':
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
        elif col_type.name == 'category':
            df[col] = df[col].cat.as_ordered()
            
        print(f'Column: {col}\nDtype before: {col_type}\nDtype after: {df[col].dtype}')
        print(30 * '-')
            
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
c = reduce_mem_usage(c)


In [ ]:
import gc
print(train.shape)
# Anti Join
outer_join = train.merge(c[c.sales == 0].drop("sales",axis = 1), how = 'outer', indicator = True)
train = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)
del outer_join
gc.collect()
train.shape

## Preenchendo previsão com 0 das lojas que nunca venderam.

In [ ]:
zero_prediction = []
for i in range(0,len(c)):
    zero_prediction.append(
        pd.DataFrame({
            "date":pd.date_range("2017-08-16", "2017-08-31").tolist(),
            "store_nbr":c.store_nbr.iloc[i],
            "family":c.family.iloc[i],
            "sales":0
        })
    )
zero_prediction = pd.concat(zero_prediction)
del c
gc.collect()
zero_prediction

## Analisando vendas pela família de produtos.

In [ ]:
a = train.groupby("family").sales.mean().sort_values(ascending = False).reset_index()
px.bar(a, y = "family", x="sales", color = "family", title = "Which product family preferred more?")

### Analisando se existe correlação com vendas e promoção.

In [ ]:
print("Spearman Correlation between Sales and Onpromotion: {:,.4f}".format(train.corr("spearman").sales.loc["onpromotion"]))

Certo, as promoções promovem as vendas, mas precisamos saber se as lojas podem ser muito diferentes entre si.

In [ ]:
d = pd.merge(train, stores)
d["store_nbr"] = d["store_nbr"].astype("int8")
d["year"] = d.date.dt.year
px.line(d.groupby(["city", "year"]).sales.mean().reset_index(), x = "year", y = "sales", color = "city")

What a mess! Probably, you are confused due to the holidays and events data. It contains a lot of information inside but, don't worry. You just need to take a breathe and think! It is a meta-data so you have to split it logically and make the data useful.

What are our problems?

Some national holidays have been transferred.
There might be a few holidays in one day. When we merged all of data, number of rows might increase. We don't want duplicates.
What is the scope of holidays? It can be regional or national or local. You need to split them by the scope.
Work day issue
Some specific events
Creating new features etc.
End of the section, they won't be a problem anymore!

In [ ]:
holidays_events["date"] = pd.to_datetime(holidays_events.date)

# holidays[holidays.type == "Holiday"]
# holidays[(holidays.type == "Holiday") & (holidays.transferred == True)]

# Transferred Holidays
tr1 = holidays_events[(holidays_events.type == "Holiday") & (holidays_events.transferred == True)].drop("transferred", axis = 1).reset_index(drop = True)
tr2 = holidays_events[(holidays_events.type == "Transfer")].drop("transferred", axis = 1).reset_index(drop = True)
tr = pd.concat([tr1,tr2], axis = 1)
tr = tr.iloc[:, [5,1,2,3,4]]

holidays = holidays_events[(holidays_events.transferred == False) & (holidays_events.type != "Transfer")].drop("transferred", axis = 1)
holidays = holidays.append(tr).reset_index(drop = True)


# Additional Holidays
holidays["description"] = holidays["description"].str.replace("-", "").str.replace("+", "").str.replace('\d+', '')
holidays["type"] = np.where(holidays["type"] == "Additional", "Holiday", holidays["type"])

# Bridge Holidays
holidays["description"] = holidays["description"].str.replace("Puente ", "")
holidays["type"] = np.where(holidays["type"] == "Bridge", "Holiday", holidays["type"])

 
# Work Day Holidays, that is meant to payback the Bridge.
work_day = holidays[holidays.type == "Work Day"]  
holidays = holidays[holidays.type != "Work Day"]  


# Split

# Events are national
events = holidays[holidays.type == "Event"].drop(["type", "locale", "locale_name"], axis = 1).rename({"description":"events"}, axis = 1)

holidays = holidays[holidays.type != "Event"].drop("type", axis = 1)
regional = holidays[holidays.locale == "Regional"].rename({"locale_name":"state", "description":"holiday_regional"}, axis = 1).drop("locale", axis = 1).drop_duplicates()
national = holidays[holidays.locale == "National"].rename({"description":"holiday_national"}, axis = 1).drop(["locale", "locale_name"], axis = 1).drop_duplicates()
local = holidays[holidays.locale == "Local"].rename({"description":"holiday_local", "locale_name":"city"}, axis = 1).drop("locale", axis = 1).drop_duplicates()



d = pd.merge(train.append(test), stores)
d["store_nbr"] = d["store_nbr"].astype("int8")


# National Holidays & Events
#d = pd.merge(d, events, how = "left")
d = pd.merge(d, national, how = "left")
# Regional
d = pd.merge(d, regional, how = "left", on = ["date", "state"])
# Local
d = pd.merge(d, local, how = "left", on = ["date", "city"])

# Work Day: It will be removed when real work day colum created
d = pd.merge(d,  work_day[["date", "type"]].rename({"type":"IsWorkDay"}, axis = 1),how = "left")

# EVENTS
events["events"] =np.where(events.events.str.contains("futbol"), "Futbol", events.events)

def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = df.select_dtypes(["category", "object"]).columns.tolist()
    # categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    df.columns = df.columns.str.replace(" ", "_")
    return df, df.columns.tolist()

events, events_cat = one_hot_encoder(events, nan_as_category=False)
events["events_Dia_de_la_Madre"] = np.where(events.date == "2016-05-08", 1,events["events_Dia_de_la_Madre"])
events = events.drop(239)

d = pd.merge(d, events, how = "left")
d[events_cat] = d[events_cat].fillna(0)

# New features
d["holiday_national_binary"] = np.where(d.holiday_national.notnull(), 1, 0)
d["holiday_local_binary"] = np.where(d.holiday_local.notnull(), 1, 0)
d["holiday_regional_binary"] = np.where(d.holiday_regional.notnull(), 1, 0)

# 
d["national_independence"] = np.where(d.holiday_national.isin(['Batalla de Pichincha',  'Independencia de Cuenca', 'Independencia de Guayaquil', 'Independencia de Guayaquil', 'Primer Grito de Independencia']), 1, 0)
d["local_cantonizacio"] = np.where(d.holiday_local.str.contains("Cantonizacio"), 1, 0)
d["local_fundacion"] = np.where(d.holiday_local.str.contains("Fundacion"), 1, 0)
d["local_independencia"] = np.where(d.holiday_local.str.contains("Independencia"), 1, 0)


holidays, holidays_cat = one_hot_encoder(d[["holiday_national","holiday_regional","holiday_local"]], nan_as_category=False)
d = pd.concat([d.drop(["holiday_national","holiday_regional","holiday_local"], axis = 1),holidays], axis = 1)

he_cols = d.columns[d.columns.str.startswith("events")].tolist() + d.columns[d.columns.str.startswith("holiday")].tolist() + d.columns[d.columns.str.startswith("national")].tolist()+ d.columns[d.columns.str.startswith("local")].tolist()
d[he_cols] = d[he_cols].astype("int8")

d[["family", "city", "state", "type"]] = d[["family", "city", "state", "type"]].astype("category")

del holidays, holidays_cat, work_day, local, regional, national, events, events_cat, tr, tr1, tr2, he_cols
gc.collect()

d.head(10)

In [ ]:
# Time Related Features
def create_date_features(df):
    df['month'] = df.date.dt.month.astype("int8")
    df['day_of_month'] = df.date.dt.day.astype("int8")
    df['day_of_year'] = df.date.dt.dayofyear.astype("int16")
    df['week_of_month'] = (df.date.apply(lambda d: (d.day-1) // 7 + 1)).astype("int8")
    df['week_of_year'] = (df.date.dt.weekofyear).astype("int8")
    df['day_of_week'] = (df.date.dt.dayofweek + 1).astype("int8")
    df['year'] = df.date.dt.year.astype("int32")
    df["is_wknd"] = (df.date.dt.weekday // 4).astype("int8")
    df["quarter"] = df.date.dt.quarter.astype("int8")
    df['is_month_start'] = df.date.dt.is_month_start.astype("int8")
    df['is_month_end'] = df.date.dt.is_month_end.astype("int8")
    df['is_quarter_start'] = df.date.dt.is_quarter_start.astype("int8")
    df['is_quarter_end'] = df.date.dt.is_quarter_end.astype("int8")
    df['is_year_start'] = df.date.dt.is_year_start.astype("int8")
    df['is_year_end'] = df.date.dt.is_year_end.astype("int8")
    # 0: Winter - 1: Spring - 2: Summer - 3: Fall
    df["season"] = np.where(df.month.isin([12,1,2]), 0, 1)
    df["season"] = np.where(df.month.isin([6,7,8]), 2, df["season"])
    df["season"] = pd.Series(np.where(df.month.isin([9, 10, 11]), 3, df["season"])).astype("int8")
    return df
d = create_date_features(d)




# Workday column
d["workday"] = np.where((d.holiday_national_binary == 1) | (d.holiday_local_binary==1) | (d.holiday_regional_binary==1) | (d['day_of_week'].isin([6,7])), 0, 1)
d["workday"] = pd.Series(np.where(d.IsWorkDay.notnull(), 1, d["workday"])).astype("int8")
d.drop("IsWorkDay", axis = 1, inplace = True)

# Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. 
# Supermarket sales could be affected by this.
d["wageday"] = pd.Series(np.where((d['is_month_end'] == 1) | (d["day_of_month"] == 15), 1, 0)).astype("int8")

d.shape

In [ ]:
d = reduce_mem_usage(d)

In [ ]:
d.info()

# Aplicando o modelo ao XGB

## Descobrindo parâmetros corretos

In [ ]:
d.tail()

In [ ]:
print(d['dcoilwtico'].dtype)
print(d['dcoilwtico'].isna().sum())
d['dcoilwtico'] = d['dcoilwtico'].astype('float32')


In [ ]:
# Preencher NaNs na coluna 'dcoilwtico' com o último valor válido
d['dcoilwtico']=d['dcoilwtico'].fillna(method='ffill')
d['dcoilwtico']=d['dcoilwtico'].fillna(method='bfill')

# Excluir a coluna 'dcoilwtico_interpolated'
d.drop('dcoilwtico_interpolated', axis=1, inplace=True)

# Preencher NaNs na coluna 'earthquake_effect' com 0
d['earthquake_effect'].fillna(0, inplace=True)


In [ ]:
# Separate the data into train+validation set and forecast set based on 'sales' column
train_valid_df = d[d['sales'].notna()]
forecast_df = d[d['sales'].isna()]

In [ ]:
# Split the data into training and validation sets
split_date = '2017-01-01'  # specify the split date in format 'YYYY-MM-DD'
train_df = train_valid_df.loc[train_valid_df['date'] <= split_date]
valid_df = train_valid_df.loc[train_valid_df['date'] > split_date]


In [ ]:
d.info()

In [ ]:
# Initialize the sum of MAPE scores to zero
total_mape_sum = 0

# Initialize the counter for combinations of store_nbr and family
num_combinations = 0

In [ ]:
exog_cols = [col for col in d.columns if col not in ['id', 'sales', 'date', 'store_nbr', 'family', 'city', 'state', 'type']]


In [ ]:
import xgboost as xgb


# Initialize an empty DataFrame to store final predictions
final_predictions = pd.DataFrame()

# Loop through each unique store
for store in d['store_nbr'].unique():
    
    # Find the unique families associated with the current store
    unique_families_for_store = d[d['store_nbr'] == store]['family'].unique()
    
    # Loop through each unique family for the current store
    for family in unique_families_for_store:
        
        # Filter the data for the current store and family
        temp_df = d[(d['store_nbr'] == store) & (d['family'] == family)]
        
        # Separate rows for training (where 'sales' is not NaN) and for prediction (where 'sales' is NaN)
        train_data = temp_df.dropna(subset=['sales'])
        prediction_data = temp_df[temp_df['sales'].isna()]
        
        # Check if train_data has data
        if len(train_data) == 0:
            print(f'No training data for Store {store}, Family {family}. Skipping...')
            continue
        
        # Separate features and target in training data
        X = train_data[exog_cols]
        y = train_data['sales']
        
        # Split the training data into train and validation sets
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Initialize and train the XGBoost model
        reg = xgb.XGBRegressor(
            objective='reg:squarederror',
            n_estimators=50,
            max_depth=5,
            learning_rate=0.1,
            colsample_bytree=0.8
        )
        
        reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=10, verbose=False)
        
        # Prepare the features for the rows with NaN sales
        X_predict = prediction_data[exog_cols]
        
        # Predict the NaN sales values using the trained model
        final_predictions_for_family = reg.predict(X_predict)
        
        # Add the predictions to the prediction_data DataFrame
        prediction_data['sales'] = final_predictions_for_family
        
        # Append this prediction_data to the final_predictions DataFrame
        final_predictions = pd.concat([final_predictions, prediction_data])

# At this point, the final_predictions DataFrame contains the rows with the NaN sales that have been filled in.
# Merge this back to the original data if needed.


In [ ]:
final_predictions.set_index('id').sort_index()

In [ ]:
# which is non-NaN only where we had a match in zero_prediction
merged_predictions = pd.merge(
    final_predictions.reset_index(), 
    zero_prediction, 
    on=['date', 'store_nbr', 'family'], 
    how='left', 
    suffixes=('', '_zero')
)

# Step 2: Update 'sales' in final_predictions to be 0 
# wherever a match was found in zero_prediction
merged_predictions.loc[merged_predictions['sales_zero'].notna(), 'sales'] = 0

# Step 3: Clean up the merged DataFrame to remove the extra columns
final_clean_predictions = merged_predictions.drop(columns=['sales_zero'])

# Step 4: Fill missing 'sales' values with 0, if any
final_clean_predictions['sales'].fillna(0, inplace=True)

# Step 5: Restore the original index of final_predictions
final_clean_predictions.set_index('id', inplace=True)

# Display the updated predictions
# Display the updated predictions
final_clean_predictions=final_clean_predictions.sort_index()

In [ ]:
final_clean_predictions

In [ ]:
# Only keep 'id' and 'sales' columns
final_clean_predictions = final_clean_predictions[['sales']]
# Create a DataFrame with a complete range of index values
full_index_range = pd.DataFrame({
    'id': np.arange(3000888, 3029400),  # 3029400 is exclusive
    'predicted_sales': 0
})

# Merge the DataFrames
final_output = pd.merge(
    full_index_range, 
    final_clean_predictions, 
    left_on='id', 
    right_index=True, 
    how='left'
)

# For indices that were in full_index_range but not in final_clean_predictions, 
# the 'predicted_sales' column from final_clean_predictions will be NaN. 
# We fill these NaNs with 0.
final_output['predicted_sales'].fillna(0, inplace=True)

# Set 'id' as the index
final_output.set_index('id', inplace=True)

# Display the updated predictions
print(final_output)
final_output['sales']=final_output['sales'].fillna(0)

In [ ]:
final_output=final_output[['sales']]

In [ ]:
final_output.to_csv('submission_kaggle.csv')

# Observando Fator de inflação da variância:

In [ ]:
# Calculate VIF for each explanatory variable
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

# Inspect the values
vif_data.head(50)

# Usando Bayes Optimization to figure out what is the best paramters for each family:

# Eficácia piorou

In [ ]:
from bayes_opt import BayesianOptimization


def xgb_eval(learning_rate, max_depth, n_estimators, colsample_bytree, subsample):
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        learning_rate=max(learning_rate, 0),
        max_depth=int(max_depth),
        n_estimators=int(n_estimators),
        colsample_bytree=max(min(colsample_bytree, 1), 0),
        subsample=max(min(subsample, 1), 0),
        random_state=42
    )
    
    model.fit(
        X_train, 
        y_train, 
        eval_set=[(X_valid, y_valid)], 
        eval_metric='rmse', 
        early_stopping_rounds=10, 
        verbose=False
    )
    
    # Here is how you access the evaluation results
    evals_result = model.evals_result()
    best_score = evals_result['validation_0']['rmse'][-1]
    
    return -best_score

def tune_and_predict(store, family, data, exog_cols):
    temp_df = data[(data['store_nbr'] == store) & (data['family'] == family)]
    train_data = temp_df.dropna(subset=['sales'])
    prediction_data = temp_df[temp_df['sales'].isna()]
    
    if len(train_data) == 0:
        print(f'No training data for Store {store}, Family {family}. Skipping...')
        return None

    global X_train, X_valid, y_train, y_valid
    X = train_data[exog_cols]
    y = train_data['sales']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)
    
    bounds = {
        'learning_rate': (0.01, 0.3),
        'max_depth': (3, 8),
        'n_estimators': (10, 100),
        'colsample_bytree': (0.7, 1),
        'subsample': (0.7, 1)
    }

    optimizer = BayesianOptimization(f=xgb_eval, pbounds=bounds, random_state=42)
    optimizer.maximize(init_points=3, n_iter=5)
    
    best_params = optimizer.max['params']
    best_params['max_depth'] = int(best_params['max_depth'])
    best_params['n_estimators'] = int(best_params['n_estimators'])

    model = xgb.XGBRegressor(objective='reg:squarederror', **best_params, random_state=42)
    model.fit(X_train, y_train)
    
    X_predict = prediction_data[exog_cols]
    prediction_data['sales'] = model.predict(X_predict)
    
    return prediction_data

# Example usage:
final_predictions = pd.DataFrame()

for store in d['store_nbr'].unique():
    unique_families_for_store = d[d['store_nbr'] == store]['family'].unique()
    for family in unique_families_for_store:
        predictions = tune_and_predict(store, family, d, exog_cols)
        if predictions is not None:
            final_predictions = pd.concat([final_predictions, predictions])


In [ ]:
final_predictions